In [1]:
from ipycanvas import Canvas

In [2]:
import dinkum
from dinkum.vfg import Gene, Receptor
from dinkum.vfn import Tissue
from dinkum import Timecourse
from dinkum import observations

In [3]:
class MultiTissuePanel:
    def __init__(self, *, states=None, tissue_names=None):
        self.panels = [ TissueActivityPanel(states=states, tissue_name=t) for t in tissue_names ]
        
    def draw(self, is_active_fn):
        total_width = 0
        x_offsets = [0]
        max_height = 0
        for p in self.panels:
            width, height = p.estimate_panel_size()
            max_height = max(height, max_height)
            total_width += width
            x_offsets.append(width)

        canvas = Canvas(width=total_width, height=max_height)
        for p, x_offset in zip(self.panels, x_offsets):
            p.draw_tissue(canvas, is_active_fn, x_offset=x_offset)
            
        return canvas

        
class TissueActivityPanel:
    box_size = 25
    box_spacing = 5
    
    box_x_start = 100
    box_y_start = 50
    
    active_color = "DeepSkyBlue"
    inactive_color = "DarkGrey"
    
    def __init__(self, *, states=None, tissue_name=None):
        assert tissue_name is not None
        self.tissue_name = tissue_name
    
        times = []
        all_gene_names = set()
        for (tp, state) in states:
            times.append(tp)
            
            activity = state.get_by_tissue_name(tissue_name)
            all_gene_names.update(activity.genes_by_name)
        self.gene_names = list(sorted(all_gene_names))

        self.times = times
        self.states = states

    def estimate_panel_size(self):
        height = len(self.times) * (self.box_size + self.box_spacing) + self.box_y_start
        width = len(self.gene_names) * (self.box_size + self.box_spacing) + self.box_x_start
        return width, height
    
    def draw(self, is_active):
        "Create canvas & draw single tissue."
        width, height = self.estimate_panel_size()
        canvas = Canvas(width=width, height=height)
        self.draw_tissue(canvas, is_active)
        
        return canvas

    def draw_tissue(self, canvas, is_active_fn, *, x_offset=0):
        "Draw this tissue on existing canvas."
        gene_names = self.gene_names
        times = self.times

        box_total_size = self.box_size + self.box_spacing

        for row in range(0, len(times)):
            for col in range(0, len(gene_names)):
                if is_active_fn(self.tissue_name, times[row], gene_names[col]):
                    canvas.fill_style = self.active_color
                else:
                    canvas.fill_style = self.inactive_color

                xpos = self.box_x_start + box_total_size*col + x_offset
                ypos = self.box_y_start + box_total_size*row
                canvas.fill_rect(xpos, ypos, self.box_size, self.box_size)

        canvas.font = "18px Arial"
        canvas.text_baseline = "top"
        canvas.fill_style = "black"

        # row names / time points
        canvas.text_align = "right"
        for row in range(0, len(times)):
            xpos = self.box_x_start - box_total_size / 2 + x_offset
            ypos = self.box_y_start + box_total_size*row
            canvas.fill_text(times[row], xpos, ypos)

        # col names / genes
        canvas.text_align = "center"
        for col in range(0, len(gene_names)):
            ypos = self.box_y_start - box_total_size
            xpos = self.box_x_start + box_total_size*col + box_total_size / 2 + x_offset

            canvas.fill_text(gene_names[col], xpos, ypos, max_width = box_total_size)

In [4]:
def tc_record_activity(*, start=1, stop=10, gene_names=None, verbose=False):    
    tc = dinkum.Timecourse()

    state_record = []     # (tp_name, state)

    time_points = {}      # time_point_name => index
    all_tissues = set()   # all tissues across all time points

    for n, state in enumerate(tc.iterate(start=start, stop=stop)):
        tp = f"t={state.time}"
        if verbose:
            print(tp)
        time_points[tp] = n

        for ti in state.tissues:
            all_tissues.add(ti.name)
            present = state[ti]
            if verbose:
                print(f"\ttissue={ti.name}, {present.report_activity()}")

        state_record.append((tp, state))

    def is_active(tissue_name, time_point, gene):
        time_idx = time_points[time_point]
        ga = states[time_idx]
        active = ga[1].get_by_tissue_name(tissue_name).is_active(gene)
        return bool(active)
    
    return state_record, list(all_tissues), is_active

## Coherent feed forward

Gene Z is turned on only once both X and Y are on, inducing a delay.

In [5]:
dinkum.reset()

# set it all up!                                                            
x = Gene(name='X')
y = Gene(name='Y')
z = Gene(name='Z')

y.activated_by(source=x)
z.activated_by_and(sources=[x, y])

m = Tissue(name='M')
x.is_present(where=m, start=1)

states, tissues, is_active_fn = tc_record_activity(stop=5)

#panel = TissueActivityPanel(states=states, tissue_name='M')
mp = MultiTissuePanel(states=states, tissue_names=['M'])
mp.draw(is_active_fn)

Canvas(height=200, width=190)

## Incoherent feed forward

Here gene Z is on for only a brief period of time.

In [6]:
dinkum.reset()

x = Gene(name='X')
y = Gene(name='Y')
z = Gene(name='Z')

y.activated_by(source=x)
z.and_not(activator=x, repressor=y)

m = Tissue(name='M')
x.is_present(where=m, start=1)

# run!                                                                      
states, tissues, is_active_fn = tc_record_activity(stop=5)
mp = MultiTissuePanel(states=states, tissue_names=['M'])
mp.draw(is_active_fn)


Canvas(height=200, width=190)

## Incoherent feed forward in two tissues

Here, gene S switches specification state b/t the two tissues.

Note that the pulse of Z should not be happening ;).

In [7]:
dinkum.reset()

# set it all up!                                                            
x = Gene(name='X')
y = Gene(name='Y')          # Y should be on in tissue N
z = Gene(name='Z')          # Z should be on in tissue M?
s = Gene(name='S')          # switches spec states b/t tissues M and N      

y.activated_by_and(sources=[x, s])    # Y requires S => on in tissue N
z.and_not(activator=x, repressor=y)   # Z is on, only when Y is not around (so, not S)

m = Tissue(name='M')
x.is_present(where=m, start=1)

m = Tissue(name='N')
x.is_present(where=m, start=1)
s.is_present(where=m, start=1)

# run!                                                                      
states, tissues, is_active_fn = tc_record_activity(stop=5)
mp = MultiTissuePanel(states=states, tissue_names=['M', 'N'])
mp.draw(is_active_fn)

Canvas(height=200, width=410)

## Mutual repression

Either A is present (and then X is on) or B is present (and Y is on)

In [8]:
dinkum.reset()

# set it all up!                                                            
x = Gene(name='X')
y = Gene(name='Y')
a = Gene(name='A')
b = Gene(name='B')

x.and_not(activator=a, repressor=y)
y.and_not(activator=b, repressor=x)

m = Tissue(name='M')
a.is_present(where=m, start=1)

n = Tissue(name='N')
b.is_present(where=n, start=1)

# run!                                                                      
states, tissues, is_active_fn = tc_record_activity(stop=5)
mp = MultiTissuePanel(states=states, tissue_names=['M', 'N'])
mp.draw(is_active_fn)

Canvas(height=200, width=380)

## Toggle switch

When co-factor is present, X is repressed; otherwise, X is expressed.

In [9]:
dinkum.reset()

# set it all up!                                                            
x = Gene(name='X')          # target of toggle switch                       
t = Gene(name='T')          # toggle switch t.f.                            
cofactor = Gene(name='cofactor')    # janus factor                          

x.toggle_repressed(tf=t, cofactor=cofactor)

# tissue M: t is present, cofactor is not; activate.                        
m = Tissue(name='M')
t.is_present(where=m, start=1)

# tissue N: t is present, along with cofactor; repress.                     
n = Tissue(name='N')
t.is_present(where=n, start=1)
cofactor.is_present(where=n, start=1)

# run!                                                                      
states, tissues, is_active_fn = tc_record_activity(stop=5)
mp = MultiTissuePanel(states=states, tissue_names=['M', 'N'])
mp.draw(is_active_fn)

Canvas(height=200, width=350)

## Community effect

An initial pulse of A in one cell results in eventual signalling-based lock-on of Y in territory.

In [10]:
dinkum.reset()

# two tissues                                                               
m = Tissue(name='M')
n = Tissue(name='N')

# neighbors                                                                 
m.add_neighbor(neighbor=n)
n.add_neighbor(neighbor=m)
assert n in m.neighbors     # should this be bidirectional? probably. CTB.  
assert m in n.neighbors

# VFN:                                                                      
a = Gene(name='A')          # transient input in M to turn on ligand L      
m.add_gene(gene=a, start=1, duration=6)

b = Gene(name='B')          # permanent input in M and N to turn on receptor R                                                                             
m.add_gene(gene=b, start=1)
n.add_gene(gene=b, start=1)

# VFG:                                                                      
ligand = Gene(name='L')          # ligand                                   
r = Receptor(name='R')      # receptor                                      
y = Gene(name='Y')          # activated by R                                

ligand.activated_or(sources=[a, y])

r.ligand(activator=b, ligand=ligand) # expression driven by B,              
                                     # activated by ligand                  
y.activated_by(source=r)    # transcription of Y turned on by activated R.  

# run!                                                                      
states, tissues, is_active_fn = tc_record_activity(stop=10)
mp = MultiTissuePanel(states=states, tissue_names=['M', 'N'])
mp.draw(is_active_fn)

Canvas(height=350, width=470)